# <p style="text-align:center"> **GIAI ĐOẠN 3: PHÂN TÍCH DỮ LIỆU** </p>
---
- **Về công cụ**: Sử dụng numpy và pandas, plotly để đặt và trả lời câu hỏi được đưa ra. 
---

### <p style="text-align:center"> **I. IMPORT THƯ VIỆN CẦN THIẾT** </p>

In [487]:
# Import thư viện cần thiết để thao tác dữ liệu
import pandas as pd
import numpy as np 
pd.options.mode.chained_assignment = None

# Import thư viện để trực quan những kết quả tìm hiểu được:
# !pip install plotly (Chưa có thư viện thì gỡ comment để tải)
# !pip install cufflinks --upgrade

import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.offline import init_notebook_mode, download_plotlyjs,plot,iplot

#Lấy các biến từ pha trước vào
%store -r dfMidfielder
%store -r dfKeeper
%store -r dfForward
%store -r dfDefender

In [488]:
# Lưu trữ các biến lại để tránh bị dính các lỗi ở các giá trị chưa được khai báo 
dfMidfielder = dfMidfielder
dfKeeper = dfKeeper
dfForward = dfForward
dfDefender = dfDefender

### <p style="text-align:center"> **II. ĐẶT VÀ TRẢ LỜI CÂU HỎI CÓ Ý NGHĨA** </p>

**Ghi chú**: Các thuộc tính có mang tiền tố average sẽ được hiểu là chỉ số trung bình trong một trận đấu của một cầu thủ. Các chỉ số này sẽ được tính bằng cách lấy chỉ số gốc chia cho tổng số lần ra sân của cầu thủ đó.

##### <p style="text-align:center">  **1. CÂU HỎI THỨ NHẤT** </p>

**Mô tả câu hỏi**: Thông thường xu hướng chơi bóng của các cầu thủ sẽ được chia làm ba loại: chủ động, trung lập và bị động. Vậy như thế nào là một cầu thủ trung lập, chủ động cũng như bị động? Và nó có ý nghĩa gì?

**Ý nghĩa:** Thông qua việc phân lớp cầu thủ này, ta có thể hiểu hơn về lối chơi của các cầu thủ và có thể giúp các huấn luyện viên có các điều chỉnh chiến thuật hoặc thậm chí là quyết đinh chuyển nhượng đội hình hợp lí.

**Cách thực hiện:** Ứng với từng vị trí, chúng ta sẽ cân nhắc các thuộc tính khác nhau để xác định xu hướng chơi bóng của các cầu thủ thuộc vị trí đó. Các bước được chia cụ thể như sau: 

*Bước 1: Xét các vị trí khác nhau sẽ có thuộc tính trung bình khác nhau.*
- Với tiền đạo: `Average shots, Average crosses, Average tackles, Average interceptions`
- Với tiền vệ: `Average crosses, Average through balls, Average passes, Average tackles, Average interceptions`
- Với hậu vệ: `Average clearances off line, Average headed clearances, Average tackles, Average interceptions`
- Với thủ môn: `Average punches, Average catches, Average high claims, Average sweeper clearance`

*Bước 2: Scale giá trị theo dạng min-max.*
$$value \ scale = \frac{value}{max} $$

Trong đó:
- value: Giá trị của thuộc tính đang xét của cầu thủ       
- max: Giá trị lớn nhất của thuộc tính trong số các cầu thủ cùng vị trí
- value scale: Giá trị đã được tính toán lại

*Bước 3: Sau khi đã tính toán ra được "điểm số xu hướng chơi bóng" (Trend), thực hiện phân lớp như sau:*
- Bị động: $$value \leq median(Trend) - 0.2 $$
- Trung lập: $$median(Trend) - 0.2 \le value \leq median(Trend) + 0.2 $$
- Chủ động: $$ value > median(Trend) + 0.2 $$   

Lí do thực hiện chia phân lớp như vậy là do sự phân bổ chỉ số ở các cầu thủ bị lệch nhiều nên việc phân loại theo giá trị trung bình sẽ không khả quan.

---
##### **1.1 VỊ TRÍ TIỀN ĐẠO**

In [489]:
# Tiền đạo: 

# Tiền xử lý dữ liệu: Xóa các cầu thủ chưa ra sân: 
dfForward_copy = dfForward[dfForward.Appearances != 0]

# Lấy trung bình
dfForward_copy['Average shots'] = (dfForward_copy['Shots'] / dfForward_copy['Appearances']) 
dfForward_copy['Average crosses'] = (dfForward_copy['Crosses'] / dfForward_copy['Appearances']) 
dfForward_copy['Average tackles'] = (dfForward_copy['Tackles'] / dfForward_copy['Appearances']) 
dfForward_copy['Average interceptions'] = (dfForward_copy['Interceptions'] / dfForward_copy['Appearances']) 

# Scale giá trị về: 
dfForward_copy['Average shots'] = (dfForward_copy['Average shots'] / dfForward_copy['Average shots'].max())
dfForward_copy['Average crosses'] = (dfForward_copy['Average crosses'] / dfForward_copy['Average crosses'].max())
dfForward_copy['Average tackles'] = (dfForward_copy['Average tackles'] / dfForward_copy['Average tackles'].max())
dfForward_copy['Average interceptions'] = (dfForward_copy['Average interceptions'] / dfForward_copy['Average interceptions'].max())


# Xu hướng chơi bóng: 
dfForward_copy['Trend'] = dfForward_copy['Average shots'] + dfForward_copy['Average crosses'] + \
                    dfForward_copy['Average tackles'] + dfForward_copy['Average interceptions']

# Xóa các cột đã sử dụng để tính toán ra kết quả cuối: 
dfForward_copy.drop(['Average shots', 'Average crosses', 'Average tackles', 'Average interceptions'], inplace=True, axis=1)

# Lấy danh sách chủ động:
neutral = dfForward_copy[dfForward_copy.Trend.between\
    (dfForward_copy['Trend'].median() - 0.2, dfForward_copy['Trend'].median() + 0.2)]
passive = dfForward_copy[dfForward_copy.Trend < dfForward_copy['Trend'].median() - 0.2]
active = dfForward_copy[dfForward_copy.Trend > dfForward_copy['Trend'].median() + 0.2]

In [490]:
dfForward_copy.head()

,Player name,Appearances,Goals Overview,Wins,Losses,Headed goals,Goals with right foot,Goals with left foot,Shots,Shots on target,...,Red cards,Fouls,Offsides,Tackles,Blocked shots,Interceptions,Clearances,Headed clearance,Involvements,Trend
1,Tammy Abraham,89,26,36,33,3,20,1,164,68,...,0,56,39,25,31,12,55,45,31,0.798687
2,Adam Armstrong,52,3,8,32,0,1,2,71,23,...,0,28,4,24,19,6,10,7,7,0.818355
3,Adama Traoré,172,9,49,76,1,7,1,149,41,...,0,142,30,122,55,34,40,22,23,1.323760
6,Julián Álvarez,12,3,10,2,0,2,1,11,6,...,0,6,1,7,3,0,1,1,3,0.593530
8,Florin Andone,26,4,5,14,2,2,0,20,9,...,1,30,4,11,6,3,13,6,4,0.552783


Danh sách cầu thủ ở vị trí tiền đạo có lối chơi chủ động:

In [491]:
active

,Player name,Appearances,Goals Overview,Wins,Losses,Headed goals,Goals with right foot,Goals with left foot,Shots,Shots on target,...,Red cards,Fouls,Offsides,Tackles,Blocked shots,Interceptions,Clearances,Headed clearance,Involvements,Trend
3,Adama Traoré,172,9,49,76,1,7,1,149,41,...,0,142,30,122,55,34,40,22,23,1.323760
10,Michail Antonio,209,58,80,80,21,28,10,458,174,...,2,240,94,153,111,136,147,75,86,1.914374
11,Antony,6,3,3,1,0,0,3,20,9,...,0,2,0,8,5,3,1,0,3,2.319867
13,Pierre-Emerick Aubameyang,134,69,63,43,4,53,12,316,144,...,1,58,80,80,76,34,59,19,85,1.402199
15,Jordan Ayew,215,29,58,105,5,19,5,289,97,...,2,324,49,338,102,110,89,55,45,1.850392
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,Theo Walcott,380,78,187,107,3,63,12,651,278,...,0,117,177,243,148,169,86,33,132,1.647311
202,Ollie Watkins,86,27,32,39,4,17,6,201,93,...,1,91,55,56,55,24,57,43,37,1.274161
203,Danny Welbeck,286,56,135,85,17,26,13,488,197,...,0,172,48,274,119,170,153,92,81,1.525622
206,Willian,267,39,151,59,1,34,4,423,154,...,1,137,38,263,120,111,59,27,78,2.035526


Danh sách cầu thủ ở vị trí tiền đạo có lối chơi bị động:

In [492]:
passive

,Player name,Appearances,Goals Overview,Wins,Losses,Headed goals,Goals with right foot,Goals with left foot,Shots,Shots on target,...,Red cards,Fouls,Offsides,Tackles,Blocked shots,Interceptions,Clearances,Headed clearance,Involvements,Trend
6,Julián Álvarez,12,3,10,2,0,2,1,11,6,...,0,6,1,7,3,0,1,1,3,0.593530
8,Florin Andone,26,4,5,14,2,2,0,20,9,...,1,30,4,11,6,3,13,6,4,0.552783
9,Jaidon Anthony,10,2,3,4,1,1,0,6,5,...,0,1,1,6,1,0,5,5,2,0.537237
12,Cameron Archer,9,0,3,5,0,0,0,1,0,...,0,5,1,0,1,0,0,0,0,0.047164
14,Taiwo Awoniyi,13,3,2,8,1,2,0,15,6,...,0,6,7,5,0,1,7,4,3,0.571113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,Chris Wood,185,53,62,74,18,25,9,295,139,...,0,158,167,52,48,10,98,71,60,0.630024
209,Xande Silva,1,0,0,1,0,0,0,2,0,...,0,0,0,0,2,0,0,0,0,0.500000
211,Andrii Yarmolenko,66,8,22,27,0,0,8,53,23,...,0,33,17,26,11,9,9,4,10,0.676842
212,Nathan Young-Coombes,1,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0.000000


Danh sách cầu thủ ở vị trí tiền đạo có lối chơi trung lập:

In [493]:
neutral

,Player name,Appearances,Goals Overview,Wins,Losses,Headed goals,Goals with right foot,Goals with left foot,Shots,Shots on target,...,Red cards,Fouls,Offsides,Tackles,Blocked shots,Interceptions,Clearances,Headed clearance,Involvements,Trend
1,Tammy Abraham,89,26,36,33,3,20,1,164,68,...,0,56,39,25,31,12,55,45,31,0.798687
2,Adam Armstrong,52,3,8,32,0,1,2,71,23,...,0,28,4,24,19,6,10,7,7,0.818355
19,Patrick Bamford,84,20,25,42,3,2,15,161,68,...,0,68,33,39,29,18,56,38,30,0.956751
22,Christian Benteke,280,86,87,129,33,41,12,654,255,...,4,408,227,98,134,32,286,209,108,0.897516
37,Dominic Calvert-Lewin,170,46,64,70,19,21,5,310,133,...,0,197,64,71,54,38,132,108,57,0.946735
74,Dwight Gayle,145,26,35,75,3,16,7,178,66,...,1,85,37,64,44,28,43,26,33,0.803715
79,Wilfried Gnonto,3,0,2,1,0,0,0,0,0,...,0,1,0,4,0,0,2,1,1,0.899297
81,Gonçalo Guedes,12,1,1,7,0,1,0,19,6,...,0,10,5,7,6,1,2,0,2,1.163398
84,Erling Haaland,13,18,9,2,3,3,12,52,28,...,0,9,4,1,9,1,4,4,21,1.155648
85,Hélder Costa,48,4,17,21,0,1,3,42,13,...,0,41,9,43,14,12,10,6,9,1.174741


Trực quan số lượng cầu thủ ở vị trí tiền đạo đã phân loại cầu thủ trên biểu đố cột:

In [494]:
df_fw_category = pd.DataFrame([active.shape[0], passive.shape[0], neutral.shape[0]], 
                            columns=['count'], index=['Active', 'Passive', 'Neutral'])

df_fw_category.iplot(kind='bar', y='count',title='Tiền đạo'
                   ,xTitle='Labels',yTitle='Total', text='count', textposition='auto')


**Nhận xét:** Theo biểu đồ thể hiện, số lượng các tiền đạo chơi chủ động và bị động là tương đối giống nhau (62 và 67). Trong khi đó số lượng cầu thủ có lối chơi trung lập
chỉ chiếm 29 cầu thủ (chỉ chiếm hơn 1/6 số tiền đạo).

##### **1.2 VỊ TRÍ TIỀN VỆ**

In [495]:
# Tiền vệ:
# Tiền xử lý dữ liệu: Xóa các cầu thủ chưa ra sân: 
dfMidfielder_copy = dfMidfielder[dfMidfielder.Appearances != 0]

# Tím trung bình các thuộc tính, các cột tính thêm phục vụ cho câu 2
dfMidfielder_copy['Average through balls'] = (dfMidfielder_copy['Through balls'] / dfMidfielder_copy['Appearances']) 
dfMidfielder_copy['Average crosses'] = (dfMidfielder_copy['Crosses'] / dfMidfielder_copy['Appearances']) 
dfMidfielder_copy['Average tackles'] = (dfMidfielder_copy['Tackles'] / dfMidfielder_copy['Appearances']) 
dfMidfielder_copy['Average interceptions'] = (dfMidfielder_copy['Interceptions'] / dfMidfielder_copy['Appearances']) 
dfMidfielder_copy['Average passes'] = (dfMidfielder_copy['Passes'] / dfMidfielder_copy['Appearances']) 
dfMidfielder_copy['Average assists'] = (dfMidfielder_copy['Assists'] / dfMidfielder_copy['Appearances']) 

# Scale giá trị về: 
dfMidfielder_copy['Average through balls'] = (dfMidfielder_copy['Average through balls'] / dfMidfielder_copy['Average through balls'].max())
dfMidfielder_copy['Average crosses'] = (dfMidfielder_copy['Average crosses'] / dfMidfielder_copy['Average crosses'].max())
dfMidfielder_copy['Average tackles'] = (dfMidfielder_copy['Average tackles'] / dfMidfielder_copy['Average tackles'].max())
dfMidfielder_copy['Average interceptions'] = (dfMidfielder_copy['Average interceptions'] / dfMidfielder_copy['Average interceptions'].max())

# Xu hướng chơi bóng: 
dfMidfielder_copy['Trend'] = dfMidfielder_copy['Average through balls'] + dfMidfielder_copy['Average crosses'] + \
                        dfMidfielder_copy['Average tackles'] + dfMidfielder_copy['Average interceptions'] + dfMidfielder_copy['Average passes']

# Xóa các cột không cần thiết: 
dfMidfielder_copy.drop(['Average through balls', 'Average crosses', 'Average tackles', 'Average interceptions'], inplace=True, axis=1)

# Lấy danh sách chủ động:
neutral = dfMidfielder_copy[dfMidfielder_copy.Trend.between\
    (dfMidfielder_copy['Trend'].median() - 0.2, dfMidfielder_copy['Trend'].median() + 0.2)]
passive = dfMidfielder_copy[dfMidfielder_copy.Trend < dfMidfielder_copy['Trend'].median() - 0.2]
active = dfMidfielder_copy[dfMidfielder_copy.Trend > dfMidfielder_copy['Trend'].median() + 0.2]

Danh sách cầu thủ ở vị trí tiền vệ có lối chơi chủ động:

In [496]:
active

,Player name,Appearances,Goals overview,Wins,Losses,Headed goals,Goals with right foot,Goals with left foot,Shots,Shots on target,...,Red cards,Fouls,Offsides,Tackles,Blocked shots,Interceptions,Clearances,Average passes,Average assists,Trend
0,Brenden Aaronson,14,1,4,7,0,1,0,19,4,...,0,8,3,24,5.0,1.0,5.0,28.571429,0.142857,29.653262
1,Tyler Adams,13,0,4,6,0,0,0,2,0,...,1,25,0,52,1.0,16.0,11.0,57.076923,0.000000,58.461885
3,Allan,52,0,20,23,0,0,0,14,5,...,1,67,0,154,4.0,42.0,59.0,38.711538,0.038462,39.703159
5,Dele Alli,194,51,103,53,9,32,10,337,134,...,0,209,36,275,87.0,146.0,124.0,32.015464,0.175258,33.078703
7,Steven Alzate,43,1,10,18,0,1,0,21,6,...,0,43,4,60,10.0,32.0,28.0,29.581395,0.000000,30.222069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362,Ryan Yates,13,0,2,7,0,0,0,17,4,...,0,28,1,22,5.0,5.0,29.0,27.538462,0.076923,28.064687
366,Oleksandr Zinchenko,83,0,68,12,0,0,0,61,15,...,1,55,6,147,21.0,86.0,86.0,69.674699,0.096386,70.791203
368,Martin Ødegaard,63,14,40,16,1,2,11,98,38,...,0,40,4,70,40.0,15.0,20.0,41.380952,0.126984,42.477416
762,Josh Brownhill,78,2,21,33,0,1,1,69,16,...,0,81,2,132,22.0,151.0,94.0,33.679487,0.051282,34.853648


Danh sách cầu thủ ở vị trí tiền vệ có lối chơi bị động:

In [497]:
passive

,Player name,Appearances,Goals overview,Wins,Losses,Headed goals,Goals with right foot,Goals with left foot,Shots,Shots on target,...,Red cards,Fouls,Offsides,Tackles,Blocked shots,Interceptions,Clearances,Average passes,Average assists,Trend
2,Marc Albrighton,307,19,117,110,2,15,2,246,81,...,1,182,23,491,83.0,292.0,204.0,22.084691,0.143322,23.730230
6,Miguel Almirón,125,17,44,46,0,3,14,177,60,...,0,80,49,200,64.0,86.0,53.0,22.880000,0.032000,23.670993
9,Andreas Pereira,60,4,26,19,0,2,2,64,24,...,0,59,6,44,20.0,24.0,15.0,24.783333,0.133333,25.569445
11,Faustino Anjorin,1,0,1,0,0,0,0,1,0,...,0,0,0,0,0.0,0.0,0.0,7.000000,0.000000,7.000000
14,Joe Aribo,15,2,3,9,0,1,1,12,8,...,0,7,0,28,4.0,5.0,10.0,20.400000,0.000000,20.971269
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,Donny van de Beek,38,3,17,16,1,1,1,10,3,...,0,15,1,36,4.0,14.0,11.0,15.157895,0.026316,15.569826
346,Nikola Vlasic,31,1,10,12,0,1,0,12,4,...,0,15,4,20,5.0,11.0,10.0,13.096774,0.000000,13.537960
354,Joe Willock,98,13,38,31,2,8,3,102,36,...,0,78,3,153,26.0,59.0,38.0,16.928571,0.030612,17.635443
355,Harry Wilson,38,7,9,22,0,0,7,62,21,...,0,15,2,26,27.0,12.0,12.0,17.657895,0.000000,18.257501


Danh sách cầu thủ ở vị trí tiền vệ có lối chơi trung lập:

In [498]:
neutral

,Player name,Appearances,Goals overview,Wins,Losses,Headed goals,Goals with right foot,Goals with left foot,Shots,Shots on target,...,Red cards,Fouls,Offsides,Tackles,Blocked shots,Interceptions,Clearances,Average passes,Average assists,Trend
68,Todd Cantwell,45,6,6,33,0,5,1,47,16,...,0,37,5,66,14.0,21.0,30.0,27.022222,0.044444,27.764491
124,Phil Foden,111,31,87,13,1,8,22,177,77,...,0,48,18,70,46.0,35.0,20.0,27.072072,0.144144,27.891837
296,Nathan Redmond,264,28,67,127,1,23,3,451,150,...,1,200,54,302,130.0,132.0,95.0,26.946970,0.098485,27.853851
367,Hakim Ziyech,51,6,24,14,0,0,6,90,32,...,0,44,9,40,34.0,23.0,11.0,26.882353,0.117647,27.937266


Trực quan hóa số lượng cầu thủ ở vị trí tiền vệ lên biểu đồ:

In [499]:
df_md_category = pd.DataFrame([active.shape[0], passive.shape[0], neutral.shape[0]], 
                            columns=['count'], index=['Active', 'Passive', 'Neutral'])

df_md_category.iplot(kind='bar', y='count',title='Tiền vệ'
                   ,xTitle='Labels',yTitle='Total', text='count', textposition='auto')

**Nhận xét:** Theo biểu đồ thể hiện, số lượng các tiền vệ chơi chủ động và bị động là tương đối giống nhau (112 và 114) và chiếm đại đa số các tiền vệ. Chỉ có 4 cầu thủ có lối chơi trung lập. Điều này cho ta thấy, việc phân biệt lối chơi cụ thể sẽ là quan trọng nhất đối với vị trí tiền vệ.

##### **1.3 VỊ TRÍ HẬU VỆ** 

In [500]:
# Hậu vệ:
# Tiền xử lý dữ liệu: Xóa các cầu thủ chưa ra sân: 
dfDefender_copy = dfDefender[dfDefender.Appearances != 0]

# Tính trung bình các thuộc tính
dfDefender_copy['Average clearances off line'] = (dfDefender_copy['Clearances off line'] / dfDefender_copy['Appearances']) 
dfDefender_copy['Average headed clearances'] = (dfDefender_copy['Headed clearance'] / dfDefender_copy['Appearances']) 
dfDefender_copy['Average tackles'] = (dfDefender_copy['Tackles'] / dfDefender_copy['Appearances']) 
dfDefender_copy['Average interceptions'] = (dfDefender_copy['Interceptions'] / dfDefender_copy['Appearances']) 

# Scale giá trị về: 
dfDefender_copy['Average clearances off line'] = (dfDefender_copy['Average clearances off line'] / dfDefender_copy['Average clearances off line'].max()) * 2
dfDefender_copy['Average headed clearances'] = (dfDefender_copy['Average headed clearances'] / dfDefender_copy['Average headed clearances'].max()) * 2
dfDefender_copy['Average tackles'] = (dfDefender_copy['Average tackles'] / dfDefender_copy['Average tackles'].max()) * 3
dfDefender_copy['Average interceptions'] = (dfDefender_copy['Average interceptions'] / dfDefender_copy['Average interceptions'].max()) * 3

# Xu hướng chơi bóng: 
dfDefender_copy['Trend'] = dfDefender_copy['Average clearances off line'] + dfDefender_copy['Average headed clearances'] + \
                        dfDefender_copy['Average tackles'] + dfDefender_copy['Average interceptions']

# Xóa các cột không cần thiết: 
dfDefender_copy.drop(['Average clearances off line', 'Average headed clearances', 'Average tackles', 'Average interceptions'], inplace=True, axis=1)

# Lấy danh sách chủ động:
neutral = dfDefender_copy[dfDefender_copy.Trend.between\
    (dfDefender_copy['Trend'].median() - 0.2, dfDefender_copy['Trend'].median() + 0.2)]
passive = dfDefender_copy[dfDefender_copy.Trend < dfDefender_copy['Trend'].median() - 0.2]
active = dfDefender_copy[dfDefender_copy.Trend > dfDefender_copy['Trend'].median() + 0.2]

Danh sách cầu thủ ở vị trí hậu vệ có lối chơi chủ động:

In [501]:
active

,Player name,Appearances,Goals Overview,Wins,Losses,Clean sheets,Goals conceded,Tackles,Blocked shots,Interceptions,...,Through balls,Yellow cards,Red cards,Fouls,Offsides,Headed goals,Goals with right foot,Goals with left foot,Hit woodwork,Trend
8,Nathan Aké,178,15,73,72,41,241,285,23,228,...,4,19,1,116,8,8,1,6,5,4.386479
11,Alex Telles,30,0,16,8,5,37,62,4,42,...,3,1,0,27,2,0,0,0,3,4.209092
13,Marcos Alonso,179,26,106,39,63,164,342,90,265,...,10,35,1,162,21,4,3,19,13,4.185895
15,Ethan Ampadu,26,0,5,20,2,42,58,1,49,...,0,3,0,25,1,0,0,0,0,4.992386
22,Serge Aurier,84,5,42,21,18,84,240,11,110,...,3,5,2,105,10,2,2,1,0,4.884119
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320,Joel Ward,265,4,84,119,63,367,586,27,447,...,3,44,0,271,6,2,1,1,3,5.194711
329,Joe Worrall,11,0,3,6,2,15,20,0,10,...,0,4,0,13,0,0,0,0,0,4.024873
331,Zanka,78,4,15,42,15,115,122,5,164,...,2,14,0,90,3,2,1,1,0,5.111071
333,Kurt Zouma,203,11,86,74,52,230,191,22,296,...,0,14,1,120,6,10,1,0,4,3.860379


Danh sách cầu thủ ở vị trí hậu vệ có lối chơi bị động:

In [502]:
passive

,Player name,Appearances,Goals Overview,Wins,Losses,Clean sheets,Goals conceded,Tackles,Blocked shots,Interceptions,...,Through balls,Yellow cards,Red cards,Fouls,Offsides,Headed goals,Goals with right foot,Goals with left foot,Hit woodwork,Trend
4,Nayef Aguerd,1,0,0,1,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,2.608063
5,Rayan Aït-Nouri,54,2,14,27,10,59,111,8,40,...,1,6,0,38,5,1,0,1,0,2.982136
6,Kristoffer Ajer,30,1,10,14,6,44,38,5,31,...,2,5,0,15,0,0,1,0,0,3.228412
7,Manuel Akanji,8,0,6,2,2,8,13,4,5,...,0,1,0,8,0,0,0,0,0,2.512393
12,Trent Alexander-Arnold,175,11,120,23,58,142,268,67,216,...,39,17,0,88,28,0,11,0,6,3.262303
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,Jake Vokins,2,0,0,1,0,3,2,0,0,...,0,0,0,2,1,0,0,0,0,0.882353
322,Kelland Watts,1,0,0,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0.882353
323,Adam Webster,95,6,30,30,22,116,95,8,122,...,4,14,0,58,1,4,1,1,3,3.290730
330,Ashley Young,404,49,189,107,83,421,569,161,456,...,108,75,2,424,102,0,48,1,17,2.973247


Danh sách cầu thủ hậu vệ có lối chơi trung lập:

In [503]:
neutral

,Player name,Appearances,Goals Overview,Wins,Losses,Clean sheets,Goals conceded,Tackles,Blocked shots,Interceptions,...,Through balls,Yellow cards,Red cards,Fouls,Offsides,Headed goals,Goals with right foot,Goals with left foot,Hit woodwork,Trend
1,Tosin Adarabioyo,43,1,9,19,10,61,48,3,53,...,0,1,0,22,2,1,0,0,0,3.833865
16,Joachim Andersen,78,2,20,27,23,88,79,4,94,...,4,13,1,72,2,1,1,0,1,3.690424
38,Ryan Bertrand,262,7,92,109,67,351,401,34,362,...,9,36,2,241,13,1,2,4,3,3.586933
70,Timothy Castagne,68,4,30,24,16,87,147,4,72,...,1,2,0,47,4,2,1,1,1,3.774940
73,Trevoh Chalobah,28,3,16,3,12,17,52,1,27,...,3,3,0,23,0,2,1,0,1,3.686265
87,Vladimír Coufal,71,0,31,25,14,88,132,7,78,...,0,7,1,65,2,0,0,0,2,3.594145
98,Eric Dier,252,12,143,58,81,250,337,43,324,...,22,38,0,197,7,6,5,1,5,3.736620
102,Matt Doherty,115,10,55,33,25,101,180,24,144,...,8,8,1,94,6,4,2,4,2,3.591682
110,Emerson Royal,43,1,24,12,7,41,96,6,31,...,1,6,1,39,2,0,1,0,0,3.468800
131,Ben Godfrey,85,0,23,47,17,136,125,9,73,...,2,13,1,47,4,0,0,0,2,3.475255


Thể hiện số lượng loại lối chơi của cầu thủ vị trí hậu vệ lên biểu đồ:

In [504]:
df_dfe_category = pd.DataFrame([active.shape[0], passive.shape[0], neutral.shape[0]], 
                            columns=['count'], index=['Active', 'Passive', 'Neutral'])

df_dfe_category.iplot(kind='bar', y='count',title='Hậu vệ'
                   ,xTitle='Labels',yTitle='Total', text='count', textposition='auto')

**Nhận xét:** Theo biểu đồ thể hiện, số lượng các hậu vệ chơi chủ động và bị động là tương đối giống nhau (91 và 92). Trong khi đó số lượng cầu thủ có lối chơi trung lập
chỉ chiếm 29 cầu thủ (chỉ chiếm hơn 1/10 số hậu vệ).

##### **1.4 VỊ TRÍ THỦ MÔN** 

In [505]:
# Thủ môn:
# Tiền xử lý dữ liệu: Xóa các cầu thủ chưa ra sân: 
dfKeeper_copy = dfKeeper[dfKeeper.Appearances != 0]

# Tính trung bình các thuộc tính
dfKeeper_copy['Average punches'] = (dfKeeper_copy['Punches'] / dfKeeper_copy['Appearances']) 
dfKeeper_copy['Average catches'] = (dfKeeper_copy['Catches'] / dfKeeper_copy['Appearances']) 
dfKeeper_copy['Average high claims'] = (dfKeeper_copy['High claims'] / dfKeeper_copy['Appearances']) 
dfKeeper_copy['Average sweeper clearance'] = (dfKeeper_copy['Sweeper clearances'] / dfKeeper_copy['Appearances']) 

# Scale giá trị về: 
dfKeeper_copy['Average punches'] = (dfKeeper_copy['Average punches'] / dfKeeper_copy['Average punches'].max()) * 2.5
dfKeeper_copy['Average catches'] = (dfKeeper_copy['Average catches'] / dfKeeper_copy['Average catches'].max()) * 2.5
dfKeeper_copy['Average high claims'] = (dfKeeper_copy['Average high claims'] / dfKeeper_copy['Average high claims'].max()) * 2.5
dfKeeper_copy['Average sweeper clearance'] = (dfKeeper_copy['Average sweeper clearance'] / dfKeeper_copy['Average sweeper clearance'].max()) * 2.5


# Xu hướng chơi bóng: 
dfKeeper_copy['Trend'] = dfKeeper_copy['Average punches'] + dfKeeper_copy['Average catches'] + \
                        dfKeeper_copy['Average high claims'] + dfKeeper_copy['Average sweeper clearance']

# Xóa các cột không cần thiết: 
dfKeeper_copy.drop(['Average punches', 'Average catches', 'Average high claims', 'Average sweeper clearance'], inplace=True, axis=1)

# Lấy danh sách chủ động:
neutral = dfKeeper_copy[dfKeeper_copy.Trend.between\
    (dfKeeper_copy['Trend'].median() - 0.2, dfKeeper_copy['Trend'].median() + 0.2)]
passive = dfKeeper_copy[dfKeeper_copy.Trend < dfKeeper_copy['Trend'].median() - 0.2]
active = dfKeeper_copy[dfKeeper_copy.Trend > dfKeeper_copy['Trend'].median() + 0.2]

Danh sách cầu thủ ở vị trí thủ môn có lối chơi chủ động:

In [506]:
active

,Player name,Appearances,Clean sheets,Wins,Losses,Saves,Punches,High claims,Catches,Sweeper clearances,Goals conceded,Yellow cards,Red cards,Fouls,Goals,Trend
0,Adrián,139,38,57,45,464.0,57,150,53,112,186,10,2,5,0,4.638350
8,Marcus Bettinelli,7,0,1,4,20.0,8,1,1,5,20,1,0,0,0,3.873403
11,Gavin Bazunu,15,1,3,9,38.0,8,16,1,8,27,0,0,0,0,3.695799
13,Asmir Begovic,256,63,82,106,765.0,91,400,138,181,357,12,0,5,1,5.474129
17,Jack Butland,87,21,25,36,306.0,36,84,37,73,130,0,0,1,0,4.622115
20,Scott Carson,146,30,45,66,435.0,66,161,164,23,225,6,1,4,0,5.324786
24,Karl Darlow,52,9,11,24,170.0,40,46,13,13,84,4,0,1,0,3.790846
30,Fabri,2,0,0,2,16.0,0,1,1,3,5,0,0,0,0,4.390244
33,Paulo Gazzaniga,43,6,13,20,127.0,18,36,17,31,61,1,0,1,0,4.168085
38,Lee Grant,31,9,10,12,99.0,21,24,16,21,41,1,0,0,0,4.751672


Danh sách cầu thủ ở vị trí thủ môn có lối chơi bị động:

In [507]:
passive

,Player name,Appearances,Clean sheets,Wins,Losses,Saves,Punches,High claims,Catches,Sweeper clearances,Goals conceded,Yellow cards,Red cards,Fouls,Goals,Trend
1,Alisson,150,68,104,17,341.0,67,64,21,99,118,2,1,3,1,2.912500
2,Álvaro Fernández,12,1,3,7,49.0,4,8,4,4,24,0,0,0,0,2.959011
3,Alphonse Aréola,39,9,5,21,118.0,27,23,5,5,51,2,0,1,0,2.714066
4,Kepa Arrizabalaga,86,29,45,24,186.0,25,38,12,33,101,3,0,0,0,2.182182
18,Willy Caballero,36,11,19,6,78.0,16,21,5,17,41,1,0,0,0,2.836495
27,Martin Dúbravka,127,37,47,50,379.0,52,101,20,45,171,2,0,0,0,2.945314
28,Ederson,196,97,149,25,335.0,47,86,24,118,145,12,1,6,0,2.407785
31,Fraser Forster,134,42,54,50,347.0,50,102,26,41,165,5,0,1,0,2.824976
32,Freddie Woodman,4,0,0,3,10.0,0,3,0,0,12,0,0,0,0,1.166159
48,João Virgínia,1,0,0,1,3.0,0,1,0,0,0,0,0,0,0,1.554878


Thể hiện số lượng loại lối chơi của cầu thủ vị trí thủ môn lên biểu đồ:

In [508]:
df_kp_category = pd.DataFrame([active.shape[0], passive.shape[0], neutral.shape[0]], 
                            columns=['count'], index=['Active', 'Passive', 'Neutral'])

df_kp_category.iplot(kind='bar', y='count',title='Thủ môn'
                   ,xTitle='Labels',yTitle='Total', text='count', textposition='auto')

Danh sách cầu thủ ở vị trí hậu vệ có lối chơi trung lập:

In [509]:
neutral

,Player name,Appearances,Clean sheets,Wins,Losses,Saves,Punches,High claims,Catches,Sweeper clearances,Goals conceded,Yellow cards,Red cards,Fouls,Goals,Trend
39,Vicente Guaita,136,39,46,54,402.0,82,77,31,47,180,8,0,3,0,3.094139
49,Sam Johnstone,37,6,4,22,166.0,23,19,13,14,74,1,0,1,0,3.376593
51,Loris Karius,29,13,17,4,51.0,15,14,7,17,26,1,0,2,0,3.234686
63,Jonas Lössl,71,14,12,43,193.0,43,49,9,36,117,2,0,0,0,3.335838
70,Alex McCarthy,141,29,37,63,448.0,78,131,29,35,229,6,0,1,0,3.353298
73,Illan Meslier,87,18,29,39,327.0,54,53,10,41,157,2,0,1,0,3.152275
82,Jordan Pickford,225,57,73,102,722.0,123,135,48,98,330,9,0,5,0,3.158650
86,Darren Randolph,34,7,12,15,91.0,9,33,8,16,61,1,0,0,0,3.313456


**Nhận xét:** Theo biểu đồ thể hiện, số lượng các thủ môn chơi chủ động và bị động là tương đối giống nhau (23 và 24). Trong khi đó số lượng cầu thủ có lối chơi trung lập
chỉ chiếm 8 cầu thủ (chỉ chiếm hơn 1/7 số thủ môn).

#### <p style="text-align:center"> **2. CÂU HỎI THỨ HAI** </p>

**Mô tả câu hỏi:** Các câu thủ nào có những hành vi không tốt trong các trận đấu hơn cả mức trung bình?

**Ý nghĩa:** Việc này có thể giúp chấn chỉnh lại hành vi thi đấu của cầu thủ, tránh những bất lợi không đáng có cho đội bóng trong trận đấu.

**Cách thực hiện:** Dựa vào số lượng lỗi, thẻ vàng và thẻ đỏ trung bình của cầu thủ trong một trận đấu, ta có thể biết liệu cầu thủ đó có đang mắc nhiều lỗi hơn mức trung bình hay không.

*Bước 1:* Gộp tất cả các cầu thủ thành một bộ dataframe thống nhất.

*Bước 2:* Lần lượt xét các thuộc tính `Average red cards, Average yellow cards, Average fouls`.

*Bước 3:* Đưa các thuộc tính về thang điểm 10 với các chỉ số chiếm tỷ trọng điểm khác nhau (5 cho average red cards; 3 cho average yellow cards, 2 cho average fouls): $$value \ scale = \frac{value}{max} \times portion$$

Trong đó: 
- value: Giá trị của thuộc tính đang xét của cầu thủ       
- max: Giá trị lớn nhất của thuộc tính trong số các cầu thủ cùng vị trí
- portion: Tỉ lệ điểm của giá trị tương ứng (tỉ lệ càng cao thì thuộc tính đó càng có giá trị để trả lời câu hỏi)
- value scale: Giá trị đã được tính toán lại

*Bước 4:* Sau khi tính toán, nếu cầu thủ nào có chỉ số "điểm phạm lỗi" lớn hơn 5 thì đang thực hiện nhiều lỗi hơn mức cần thiết.

---

In [510]:
# Tiền xử lý dữ liệu: Gộp 4 dataframe thành 1 dataframe thống nhất:
dfPlayers_1 = pd.merge(dfMidfielder, dfKeeper, how='outer')
dfPlayers_2 = pd.merge(dfForward, dfDefender, how='outer') 
dfPlayers = pd.merge(dfPlayers_1, dfPlayers_2, how='outer') 
dfPlayers = dfPlayers[['Player name', 'Appearances', 'Yellow cards', 'Red cards', 'Fouls']]

In [511]:
# Chỉ lấy cầu thủ ra sân:
dfPlayers = dfPlayers[dfPlayers.Appearances != 0]

# Scale về chỉ số trung bình
dfPlayers['Yellow cards'] = dfPlayers['Yellow cards'] / dfPlayers['Appearances']
dfPlayers['Red cards'] = dfPlayers['Red cards'] / dfPlayers['Appearances']
dfPlayers['Fouls'] = dfPlayers['Fouls'] / dfPlayers['Appearances']

# Scale về chỉ số tỉ lệ
dfPlayers['Yellow cards'] = (dfPlayers['Yellow cards'] / dfPlayers['Yellow cards'].max()) * 3
dfPlayers['Red cards'] = (dfPlayers['Red cards'] / dfPlayers['Red cards'].max()) * 5 
dfPlayers['Fouls'] = (dfPlayers['Fouls'] / dfPlayers['Fouls'].max()) * 2

# Tiến hành xét điểm đánh gía chung: 
dfPlayers['Score'] = dfPlayers['Yellow cards'] + dfPlayers['Red cards'] + dfPlayers['Fouls']

# Note nhỏ: Sau khi lọc ra xong rồi tính median đồ lại.

# Danh sách cầu thủ cần chấn chỉnh hành vi thi đấu: 
get_players = dfPlayers[dfPlayers['Score'] > dfPlayers['Score'].max() / 2] 
get_players

,Player name,Appearances,Yellow cards,Red cards,Fouls,Score
1,Tyler Adams,13,0.692308,2.692308,1.785714,5.170330
19,Tiemoué Bakayoko,29,0.465517,1.206897,1.600985,3.273399
125,Conor Gallagher,76,1.302632,0.460526,1.441729,3.204887
187,Jefferson Lerma,78,1.500000,0.448718,1.190476,3.139194
222,Hannibal Mejbri,3,3.000000,0.000000,0.928571,3.928571
346,Granit Xhaka,202,1.202970,0.693069,1.241160,3.137199
481,Florin Andone,26,0.865385,1.346154,1.071429,3.282967
536,Julio Enciso,2,2.250000,0.000000,1.392857,3.642857
609,Darwin Núñez,10,0.000000,3.500000,0.835714,4.335714
646,Luis Sinisterra,9,0.500000,3.888889,0.619048,5.007937


Biểu đồ minh họa các cầu thủ nằm trong top "hành vi không đúng" nhất:

In [512]:
fig = px.bar(get_players, x='Player name', y=['Red cards', 'Yellow cards', 'Fouls'], 
            title="Players need to correct their playing behavior")

fig.show()

**Nhận xét**: Việc phân tích này cho ta thấy các cầu thủ được nêu tên trong bảng đang thực hiện quá nhiều lỗi không cần thiết. Cần làm việc với các cầu thủ ở trên để chấn chỉnh lại thái độ và phong cách thi đấu của họ.

#### <p style="text-align:center"> **3. CÂU HỎI THỨ BA** </p>

Câu hỏi thứ 3 sẽ tập hợp nhiều câu hỏi nhỏ có cùng cách tính: 

##### **3.1 - Vị trí tiền đạo**

**Câu hỏi nhỏ 1**: Đối với vị trí tiền đạo, có ý kiến cho rằng cầu thủ ghi càng nhiều bàn thắng thì họ họ sẽ có xu hướng chơi bóng cá nhân hơn. Bằng việc so sánh độ tương quan giữa cột giá trị average goals với 2 cột giá trị average passes và average assists, ta sẽ biết được liệu ý kiến ở trên là đúng hay sai.

**Câu hỏi nhỏ 2**: Đối với vị trí tiền đạo, có ý kiến cho rằng cầu thủ càng bỏ lỡ nhiều cơ hội ghi bàn thì họ họ sẽ có xu hướng chơi bóng đồng đội, kiến tạo nhiều hơn. Bằng việc so sánh độ tương quan giữa cột giá trị average big chances missed với 2 cột giá trị average passes và average assists, ta sẽ biết được liệu ý kiến ở trên là đúng hay sai.

**Ý nghĩa:** Cả hai câu hỏi trên đều sẽ giúp ta có một góc nhìn toán học hơn sự liên quan giữa các chỉ số, đồng thời tạo khả năng nhìn vào một thông số nhưng lại có thể đánh giá tổng thể lối chơi của cầu thủ.

In [513]:
#CODE HERE
dfForward_copy['Average goals'] =  (dfForward_copy['Goals Overview'] / dfForward_copy['Appearances'])
dfForward_copy['Average passes'] =  (dfForward_copy['Passes'] / dfForward_copy['Appearances'])
dfForward_copy['Average assists'] =  (dfForward_copy['Assists'] / dfForward_copy['Appearances'])
dfForward_copy['Average big chances missed'] = (dfForward_copy['Big chances missed'] / dfForward_copy['Appearances'])

dfForward_corr = dfForward_copy[['Average goals','Average passes', 'Average assists', 'Average big chances missed']].copy().corr() 

fig = px.imshow(np.array(dfForward_corr), x = dfForward_corr.columns, y = dfForward_corr.index, text_auto=True)
fig.show()

**CÂU TRẢ LỜI:** Dựa vào heatmap ở trên, ta thấy các chỉ số đều tương quan dương với nhau.
+ Câu hỏi thứ nhất: Không có chuyện cầu thủ càng ghi nhiều bàn thắng thì sẽ có lối chơi cá nhân hơn mà ngược lại cầu thủ càng có nhiều bàn thắng thì các chỉ số còn lại của cầu thủ đó cũng sẽ cao theo.
+ Câu hỏi thứ hai: Nhận định này là hoàn toàn đúng. Tuy nhiên độ ảnh hưởng của nó không quá đáng kể vì độ tương quan giữa các chỉ số không quá lớn lần lượt là 0.316 và 0.274 khi so với chỉ số chuyền bóng và kiến tạo.

#### **3.2 - Vị trí hậu vệ**

**Câu hỏi nhỏ 3:** Đối với vị trí hậu vệ, có ý kiến cho rằng cầu thủ càng bị thủng lưới nhiều sẽ có xu hướng chơi bóng chủ động hơn. Bằng việc so sánh độ tương quan giữa cột giá trị average goals conceded với cột giá trị "điểm số xu hướng chơi bóng", ta sẽ biết được liệu ý kiến ở trên là đúng hay sai. 

**Ý nghĩa:** Việc này sẽ giúp công tác chuẩn bị tâm lí thi đấu với các cầu thủ sẽ được cải thiện.

**Câu hỏi nhỏ 4:** Đối với vị trí hậu vệ, có ý kiến cho rằng các cầu thủ nên giảm thiểu số lượng chuyền bóng để ít mắc lỗi hơn dẫn đến bàn thắng cho đối phương hơn. Bằng việc so sánh độ tương quan giữa cột giá trị average passes với cột giá trị average errors leading to goal, ta sẽ biết được liệu ý kiến ở trên là đúng hay sai. 

**Ý nghĩa:** Việc này sẽ giúp điều chỉnh lối đá của cầu thủ nhắm tăng khả năng phòng thủ của đội bóng.

In [514]:
dfDefender_copy['Average goals conceded'] =  (dfDefender_copy['Goals conceded'] / dfDefender_copy['Appearances'])
dfDefender_copy['Average passes'] = (dfDefender_copy['Passes'] / dfDefender_copy['Appearances'])
dfDefender_copy['Average error leading to goal'] = (dfDefender_copy['Errors leading to goal'] / dfDefender_copy['Appearances'])

dfDefender_coor = dfDefender_copy[['Average passes', 'Average error leading to goal', 
                                    'Trend', 'Average goals conceded']].copy().corr() 

fig = px.imshow(np.array(dfDefender_coor), x = dfDefender_coor.columns, y = dfDefender_coor.index, text_auto=True)
fig.show()

**CÂU TRẢ LỜI:** Dựa vào heatmap ở trên, ta thấy các chỉ số có độ tương quan không đáng kể, trừ độ tương quan giữa Average passes với Trend và Average goals conceded là khá cao.
+ Câu hỏi thứ ba: độ tương quan giữa Average goals conceded và Trend là 0.454 điều này chứng tỏ nhận định trên là đúng.
+ Câu hỏi thứ tư: Nhận định này là hoàn toàn sai do độ tương quan của hai chỉ số là không đáng kể (0.096). Điều này chứng tỏ việc số lượng đường chuyền không ảnh hưởng tới số bàn thua mà cầu thủ phải nhận.

##### **3.3 - Vị trí tiền vệ:**

**Câu hỏi 5:** Đối với vị trí tiền vệ, hầu hết chúng ta có thể nghĩ tới việc cầu thủ ra sân càng nhiều thì sẽ có nhiều kinh nghiệm và chất lượng chơi bóng của cầu thủ đó sẽ tốt hơn. Bằng việc so sánh độ tương quan giữa cột giá trị appearance với 2 cột giá trị crosses accuracy và accurate long balls, ta sẽ xác minh được quan điểm trên. 

**Ý nghĩa**: Điều này sẽ có lợi với việc quyết định giữ lại các cầu thủ gạo cội hay ưu tiên bồi dưỡng các lứa cầu thủ trẻ hơn.

**Câu hỏi 6:** Đối với vị trí tiền vệ, có ý kiến cho rằng việc những đường chuyền vượt tuyến(through ball) của các tiền vệ là những tình huống tạo ra độ nguy hiểm cao đối với đối phương. Bằng việc so sánh độ tương quan giữa cột giá trị average through balls với cột giá trị average big chances created, ta sẽ biết liệu ý kiến trên là đúng hay sai. 

**Ý nghĩa:** Điều này sẽ giúp cho các đội bóng có cách đối phó với lối chơi của từng cầu thủ tiền vệ.

In [515]:
dfMidfielder_copy['Average through balls'] = (dfMidfielder_copy ['Through balls'] / dfMidfielder_copy ['Appearances'])
dfMidfielder_copy['Average big chances created'] = (dfMidfielder_copy ['Big chances created'] / dfMidfielder_copy ['Appearances'])

dfMidfielder_coor = dfMidfielder_copy[['Appearances', 'Cross accuracy',
                                    'Average through balls', 'Average big chances created', 'Accurate long balls']].copy().corr() 


fig = px.imshow(np.array(dfMidfielder_coor), x = dfMidfielder_coor.columns, y = dfMidfielder_coor.index, text_auto=True)
fig.show()

**CÂU TRẢ LỜI:** Dựa vào heatmap ở trên, ta thấy các chỉ số độ tương quan là khá đa dạng.
+ Câu hỏi thứ năm: việc ra sân nhiều sẽ ảnh hưởng rất nhiều tới khả chơi bóng dài của cầu thủ với bằng chứng là độ tương quan giữa Apperances và Accurate long balls là rất cao (0.818). Mặt khác độ chính xác cũng được cải thiện theo số lần ra sân tuy nhiên sự tương quan không được sâu sắc như trên (0.264)
+ Câu hỏi thứ sáu: Nhận định này là hoàn toàn đúng do độ tương quan giữa hai chỉ số là khá cao (0.669)

#### <p style="text-align:center"> **5. CÂU HỎI THỨ TƯ** <p>

**Mô tả câu hỏi:** Đội hình hoàn hảo là đội hình có các vị trí bao gồm tiền đạo, tiền vệ, hậu vệ, thủ môn đều có các chỉ số tốt dựa trên thông số thống kê được. Vậy, để xây dựng một đội hình hoàn hảo thì huấn luyện viên chọn ra các thông số tốt nhất đến từ các vị trí khác nhau. Làm sao để chọn được các cầu thủ có thông số tốt nhất để xây dựng đội hình?

**Mục đích**: Các huấn luyện viên có thể dùng câu hỏi này để tuyển chọn các cầu thủ tốt nhất vào câu lạc bộ của mình khi họ hết hợp đồng với các câu lạc bộ khác. Việc chọn cầu thủ tốt có thể sẽ xây dựng nên một đội hình mạnh, xây dựng chiến thuật được dễ dàng và nhận được nhiều bàn thắng hơn.

**Cách thực hiện:**
- *Bước 1: Chọn top 5 cầu thủ ở vị trí tiền vệ. Các cầu thủ ở vị trí tiền vệ được đánh giá qua số bàn thắng (Goals overview) và tổng số lần kiên tạo của họ (Assists)* .

In [516]:
# Tiền vệ 
# Lấy ra top 5 người có bàn thắng cao nhất:
dfMidfielder_top =  dfMidfielder[dfMidfielder['Appearances'] != 0]
dfMidfielder_scores = dfMidfielder_top.nlargest(5, ['Goals overview'])

dfMidfielder_scores.iplot(kind='bar',x='Player name',y='Goals overview', title='Top Goalscorers', 
                        xTitle='Players', yTitle='Goals',color='red');

In [517]:
# Lấy ra top 5 người có chỉ số kiến tạo cao nhất:
dfMidfielder_assists = dfMidfielder_top.nlargest(5, ['Assists'])

dfMidfielder_assists.iplot(kind='bar', x='Player name', y='Assists', title='Top Assists', 
                        xTitle='Players', yTitle='Assists',color='red');
                        

**Nhận xét:** Ta vẫn thấy 4 người Kevin De Bruyne, James Milner, Christian Eriksen, Juan Mata vẫn nằm trong top bàn thắng và top kiến tạo. Vậy, để có cái nhìn toản cảnh hơn, nhóm sẽ plot biểu đồ biểu thị sự liên quan đến tiêu chí ta đang chọn (bao gồm assists và goals). 

In [518]:
dfMidfielder_top['Involvements'] = dfMidfielder_top['Goals overview'] + dfMidfielder_top['Assists']
involvements =  dfMidfielder_top.nlargest(5, 'Involvements')

# involvements.plot(kind='line', x='Player name', y='Involvements', xlabel='Name', 
#                         ylabel='Goal Involvements', color='red', figsize=(10, 10), marker='X');

involvements.iplot(x='Player name',y='Involvements',title='Players with highest Goal Involvements'
                   ,xTitle='Players',yTitle='Goal Involvements')

**Kết luận**: Như vậy, với vị trí là tiền vệ, ta có thể chọn được 4 người có chỉ số bàn thắng cũng như kiến tạo toàn diện nhất là Kevin De Bruyne, James Milner, Christian Eriksen, Juan Mata.

- *Bước 2: Với vị trí tiền đạo, ta cũng làm tương tự như vị trí tiền vệ*.

In [519]:
# Tiền đạo
# Lấy ra top 5 người có chỉ số bàn thắng cao nhất:
dfForward_top =  dfForward[dfForward['Appearances'] != 0]
dfForward_scores = dfForward_top.nlargest(5, ['Goals Overview'])

dfForward_scores.iplot(kind='bar',x='Player name',y='Goals Overview', title='Top Goalscorers', 
                        xTitle='Players', yTitle='Goals',color='red');

In [520]:
# Lấy ra top 5 người có chỉ số kiến tạo cao nhất
dfForward_assists = dfForward_top.nlargest(5, ['Assists'])

dfForward_assists.iplot(kind='bar', x='Player name', y='Assists', title='Top Assists', 
                        xTitle='Players', yTitle='Assists',color='red');

In [521]:
dfForward['Involvements'] = dfForward['Goals Overview'] + dfForward['Assists']
involvements =  dfForward.nlargest(5, 'Involvements')

involvements.iplot(x='Player name',y='Involvements',title='Players with highest Goal Involvements'
                   ,xTitle='Players',yTitle='Goal Involvements')

**Kết luận**: Từ biểu đồ đường, ta lấy 3 người đang nằm trong dốc ở mức chấp nhận được là Harry Kane, Jamie Vardy, Mohamed Salah.

- ***Bước 3**: Mặc dù ta có thể dùng thuộc tính "Clean sheets" để thực hiện cho bước này nhưng có thể nó sẽ không cho ta cái nhìn tổng quan hơn về các hậu vệ. Mặc dù "Clean sheets" nhiều hơn đồng nghĩa với việc hiệu suất phòng ngự chắc chắn hơn nhưng nó lại không cung cấp cho ta thông tin gì về số bàn thúa trong các trận đấu khác. Nhiều bàn thua hơn trong một trận đấu thì phản ánh hàng phòng thủ kém.*
  - *Do đó, ở bước 3 này, ta sẽ thực hiện dựa trên thuộc tính số bàn thua mỗi trận được tính bằng "Goals conceded" chia với số lần ra sân của cầu thủ đó. Và tất nhiên số lần ra sân phải thỏa mãn ngưỡng nào đó cho trước để đánh gía công tâm nhất.*

In [522]:
# Hậu vệ:
dfDefender_top = dfDefender[dfDefender['Appearances'] > 30]
dfDefender_top['Conceded per game'] = dfDefender_top['Goals conceded'] / dfDefender_top['Appearances']
least_conceded = dfDefender_top.nsmallest(5, 'Conceded per game')

least_conceded.iplot(kind='bubble',x='Player name',y='Conceded per game',size='Clean sheets', title='Goals conceded per game')

**Nhận xét**: Biểu đồ bong bóng ở trên thể hiện màn trình diễn xuất sắc của các cầu thủ. Với chiều rộng của bong bóng mô tả số trận giữ sạch lưới. Vì vậy, ta có thể chọn ra được các cầu thủ để thủng lưới ít bàn hơn mỗi trận và giữ sạch lưới nhiều hơn là: Aymetric, João Cancelo, Joe Gomez.

- ***Bước 4:** Với vị trí thủ môn, ta thực hiện tương tự như vị trí hậu vệ. Nhưng thay vì chiều rộng bong bóng mô tả "clean sheets" thì chiều rộng mô tả số lần giữ lưới an toàn (saves).*

In [523]:
# Thủ môn
dfKeeper_top = dfKeeper[dfKeeper['Appearances'] >= 30]
dfKeeper_top['Conceded per game'] = (dfKeeper_top['Goals conceded'] / dfKeeper_top['Appearances']) 
least_conceded = dfKeeper_top.nsmallest(5, 'Conceded per game')

least_conceded.iplot(kind='bubble',x='Player name',y='Conceded per game',size='Saves', title='Goals conceded per game')

**Nhận xét:** Từ biểu đồ, ta có thể lấy được Alisson, Hugo Lioris và Ederson.

**Kết luận cuối:**
- Với tiền vệ ta có thể chọn được các cầu thủ: **Kevin De Bruyne, James Milner, Christian Eriksen, Juan Mata.**
- Với tiền đạo ta có thể chọn được các cầu thủ: **Harry Kane, Jamie Vardy, Mohamed Salah.**
- Với hậu vệ ta có thể chọn được các cầu thủ: **Aymetric, João Cancelo, Joe Gomez.**
- Với thủ môn ta có thể chọn được các cầu thủ: **Alisson, Hugo Lioris và Ederson.**

#### <p style="text-align:center"> **6. CÂU HỎI THỨ NĂM** <p>

**Mô tả câu hỏi:**  Những thuộc tính nào phân biệt rõ ràng nhất giữ các cầu thủ chơi ở vị trí tiền đạo và vị trí tiền vệ,

**Mục đích**: Khi nhìn vào thuộc tính tìm được, ta có thể nhanh chóng nhận dạng cầu thủ đó phù hợp với vị trí mình đang chơi hay không?

**Cách thực hiện:** Tìm cách thuộc tính chung của hai vị trí tiền đạo và tiền vệ. Tìm các thuộc tính có cách biệt lớn nhất.

In [524]:
share_columns = [ i for i in dfForward_copy.columns if i in dfMidfielder_copy.columns]

In [525]:
def mean(column):
    return column.mean()

In [526]:
dfForward_mean = dfForward_copy.drop(columns=[col for col in dfForward_copy if col not in share_columns])
dfMidfielder_mean = dfMidfielder_copy.drop(columns=[col for col in dfMidfielder_copy if col not in share_columns])

In [527]:
dfForward_mean.drop(columns=['Player name'] , inplace=True)
dfMidfielder_mean.drop(columns=['Player name'] , inplace=True)

In [529]:
dfForward_mean = dfForward_mean.agg([mean])
dfMidfielder_mean = dfMidfielder_mean.agg([mean])

In [530]:
dfForward_mean

,Appearances,Wins,Losses,Headed goals,Goals with right foot,Goals with left foot,Shots,Shots on target,Hit woodwork,Assists,...,Red cards,Fouls,Offsides,Tackles,Blocked shots,Interceptions,Clearances,Trend,Average passes,Average assists
mean,79.044304,33.417722,27.639241,3.170886,10.759494,6.088608,143.765823,57.803797,4.189873,8.987342,...,0.303797,63.816456,30.556962,57.177215,36.512658,26.012658,34.531646,1.002139,13.809819,0.080754


In [531]:
dfMidfielder_mean

,Appearances,Wins,Losses,Headed goals,Goals with right foot,Goals with left foot,Shots,Shots on target,Hit woodwork,Assists,...,Red cards,Fouls,Offsides,Tackles,Blocked shots,Interceptions,Clearances,Average passes,Average assists,Trend
mean,78.13913,33.121739,27.195652,0.547826,4.591304,2.56087,86.4,27.721739,2.143478,7.513043,...,0.291304,69.813043,6.46087,120.521739,26.756522,69.082609,62.426087,27.74064,0.066811,28.55369


In [549]:
dfDiff = abs(dfForward_mean - dfMidfielder_mean)

In [551]:
dfDiff.T.sort_values(by='mean', ascending=False)[:5]

,mean
Passes,1257.900605
Tackles,63.344524
Shots,57.365823
Crosses,43.248927
Interceptions,43.069950


**Nhận xét:** Vậy các thuộc tính phân biệt một tiền đạo và một tiền vệ một cách rõ ràng nhất là Passes. Lần lượt 4 thuộc tính theo sau đó là Tackles. Shots, Crosses, Interceptions. Vậy khi nhìn vào một cầu thủ,chỉ số Passes sẽ phản án rõ ràng nhất việc cầu thủ đó có đang chơi sai vị trí sở trường của mình hay không.